In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats
import seaborn as sns
from matplotlib import rcParams

from sklearn.ensemble import RandomForestClassifier

# Complete example using RandomForestClassifier

In [3]:
# from https://github.com/benhamner/ExpediaPersonalizedSortCompetition/blob/master/PythonBenchmark/train.py
df = pd.read_csv('../../training_set_VU_DM_2014.csv').fillna(value=0)

In [4]:
# remove features not available in test set
feature_names = list(df.columns)
feature_names.remove("click_bool")
feature_names.remove("booking_bool")
feature_names.remove("gross_bookings_usd")
feature_names.remove("date_time")
feature_names.remove("position")

In [5]:
# split into features and target
features = df[feature_names].values
target = df["booking_bool"].values

In [6]:
# train model
print("Training the Classifier")
classifier = RandomForestClassifier(n_estimators=50, 
                                        verbose=2,
                                        n_jobs=4,
                                        min_samples_split=10,
                                        random_state=1)
classifier.fit(features, target)

Training the Classifier
building tree 1 of 50
building tree 2 of 50building tree 3 of 50building tree 4 of 50


building tree 5 of 50
building tree 6 of 50
building tree 7 of 50
building tree 8 of 50
building tree 9 of 50
building tree 10 of 50
building tree 11 of 50
building tree 12 of 50
building tree 13 of 50
building tree 14 of 50
building tree 15 of 50
building tree 16 of 50
building tree 17 of 50
building tree 18 of 50
building tree 19 of 50
building tree 20 of 50
building tree 21 of 50
building tree 22 of 50
building tree 23 of 50
building tree 24 of 50
building tree 25 of 50
building tree 26 of 50
building tree 27 of 50
building tree 28 of 50
building tree 29 of 50
building tree 30 of 50
building tree 31 of 50
building tree 32 of 50
building tree 33 of 50
building tree 34 of 50
building tree 35 of 50
building tree 36 of 50


[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed:  4.2min


building tree 37 of 50
building tree 38 of 50
building tree 39 of 50
building tree 40 of 50
building tree 41 of 50
building tree 42 of 50
building tree 43 of 50
building tree 44 of 50
building tree 45 of 50
building tree 46 of 50
building tree 47 of 50
building tree 48 of 50
building tree 49 of 50
building tree 50 of 50


[Parallel(n_jobs=4)]: Done  50 out of  50 | elapsed:  6.6min finished


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=10,
            min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=4,
            oob_score=False, random_state=1, verbose=2, warm_start=False)

In [7]:
# # save model
# filename = 'model.sav'
# pickle.dump(classifier, open(filename, 'wb'))

# # load model
# loaded_model = pickle.load(open(filename, 'rb'))

In [8]:
# load test set
test = pd.read_csv('../../test_set_VU_DM_2014.csv').fillna(value=0)

In [9]:
# obtain feature names and remove useless column date_time
feature_names = list(test.columns)
feature_names.remove("date_time")

# obtain feature values
features = test[feature_names].values

# predict using trained model
print("Making predictions")
predictions = classifier.predict_proba(features)[:,1]
predictions = list(-1.0*predictions)
recommendations = zip(test["srch_id"], test["prop_id"], predictions)

Making predictions


[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed:   26.2s
[Parallel(n_jobs=4)]: Done  50 out of  50 | elapsed:   35.3s finished


In [10]:
# sort based on recommendation value and write sorted prediction to file

from operator import itemgetter
import csv

rows = [(srch_id, prop_id)
        for srch_id, prop_id, rank_float
        in sorted(recommendations, key=itemgetter(0,2))]
writer = csv.writer(open("predict.csv", "w"), lineterminator="\n")
writer.writerow(("SearchId", "PropertyId"))
writer.writerows(rows)